In [1]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

C:\Users\karin\AppData\Local\Temp\ipykernel_8948\3698147250.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
def sacar_tuplas_moviedatabase(archivo):
    """
    Extrae las tuplas de datos de un archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []
    
    # Lista para almacenar las filas duplicadas
    duplicadas = []
    
     # Conjunto para almacenar los IDs de las películas ya vistas
    vistas = set()

    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
        
        # Convierte la fila en una lista
        pelicula = list(pelicula)

        # Obtiene el ID de la película
        id_pelicula = pelicula[-1]

        # Verifica si el ID de la película ya está en el conjunto de películas vistas
        if id_pelicula in vistas:
            duplicadas.append(pelicula)
        else:
            # Agrega el ID de la película al conjunto de películas vistas
            vistas.add(id_pelicula)
            val.append(tuple(pelicula))

        # Itera sobre cada elemento de la película
        for i, elemento in enumerate(pelicula):
            
            # Reemplaza los valores NaN con None
            if elemento is np.nan:
                pelicula[i] = None
            
                # Intenta convertir los elementos de tipo str a float o int si es posible
            
                if isinstance(elemento, str) and any(caracter.isdigit() for caracter in elemento):

                    try:
                        pelicula[i] = int(elemento)
                    except ValueError:
                            pass
                if isinstance(elemento, str) and "." in elemento:

                    try:
                        pelicula[i] = float(elemento)
                    except:
                        pass
                
                if isinstance(elemento, str) and any(caracter.isdigit() for caracter in elemento):

                    try:
                        pelicula[i] = int(elemento)
                    except:
                        pass
                    
                if isinstance(elemento, str) and elemento.startswith('-') and elemento.endswith('-'):

                    try:
                        pelicula[i] = None
                    except:
                        pass
                    
        # Verifica si la película ya está en la lista de datos
        for dato in val:
            if pelicula[-1] in dato:
                duplicadas.append(pelicula)
            
        # Agrega la película a la lista de datos si no está duplicada
        if pelicula not in duplicadas:
            val.append(tuple(pelicula))
    
    # Retorna la lista de tuplas
    return val

In [13]:
archivo = 'BBDD_Importaciones/Peliculas_detalles/detalles_cortos_2015_2019.csv'
prueba = sacar_tuplas_moviedatabase(archivo)

In [14]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')



# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
query = "INSERT INTO detalles (PuntuacionRottenTomatoes, PuntuacionIMDB, Direccion, Guion, Argumento, Duracion, Genero, Titulo, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"


# Si el archivo de origen contiene índices, llamar a la funcion 'sacar_tuplas_i_moviedatabase'
val = sacar_tuplas_moviedatabase(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1265 (01000): Data truncated for column 'PuntuacionIMDB' at row 1
Error Code: 1265
SQLSTATE 01000
Message Data truncated for column 'PuntuacionIMDB' at row 1
